<a href="https://colab.research.google.com/github/jgu13/Transformers_with_Multiheaded_Attention/blob/main/IFT6135_2023_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This notebook is intended to produce the plots and figures for the report on Problem 1 of the practical. You should not run this notebook in Google Colab until you have finished constructing the correct solutions for transformer_solution.py and encoder_decoder_solution.py

This notebook provides some limited commentary on several HuggingFace Features and toolage. You will use HuggingFace Datasets to load the Amazon Polarity dataset for sentiment analysis. The notebook will define a Bert tokenizer, collate functions, and then train and evaluate several models using the HuggingFace utilities mentioned above. Remember, the most crucial part here is running the experiments for the report.

In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. 
import sys
import os
import shutil
import warnings

folder = "" #@param {type:"string"}
!ln -Ts "$folder" /content/assignment 2> /dev/null

# Add the assignment folder to Python path
if '/content/assignment' not in sys.path:
  sys.path.insert(0, '/content/assignment')

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

<ipython-input-2-f8159205a9d4>:20: UserWarning: CUDA is not available.
  warnings.warn('CUDA is not available.')


### Running on GPU
For this assignment, it will be necessary to run your experiments on GPU. To make sure the notebook is running on GPU, you can change the notebook settings with
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`


In [ ]:
sys.path.append("/content/gdrive/MyDrive/Representation Learning/Practice/A2")

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.metrics import f1_score   
import time

from typing import List, Dict, Union, Optional, Tuple
import torch

from dataclasses import dataclass
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm.auto import tqdm


!pip install -qqq datasets transformers --upgrade
from datasets import Dataset
import transformers

from datasets import load_dataset
from tokenizers import Tokenizer

from transformer_solution import Transformer
from encoder_decoder_solution import EncoderDecoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.8 MB/s eta 0:00:00


In [ ]:
dataset_train = load_dataset("amazon_polarity", split="train", cache_dir="assignment/data")
dataset_test = load_dataset("amazon_polarity", split="test[:1000]", cache_dir="assignment/data")

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

Dataset amazon_polarity downloaded and prepared to /content/assignment/data/amazon_polarity/amazon_polarity/3.0.0/a27b32b7e7b88eb274a8fa8ba0f654f1fe998a87c22547557317793b5d2772dc. Subsequent calls will reuse this data.


In [ ]:
import torch
import torch.nn as nn

x_new = torch.empty((1,3,10))
hidden_states = torch.empty((2,1,10))
hidden_states = hidden_states[-1] # use the last layer
hidden_states = hidden_states.repeat(x_new.size()[1], 1, 1)
hidden_states = torch.transpose(hidden_states, 0, 1)
# hidden_states.size()
torch.cat((hidden_states, x_new), axis=2).size()

torch.Size([1, 3, 20])

In [ ]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output.size()

torch.Size([2, 3])

In [ ]:
model = nn.GRU(input_size=10, hidden_size=10, num_layers=2, batch_first=True, bidirectional=True)
inputs = torch.randn((1,3,10))
hidden_states = torch.randn((4, 1, 10)) # 4 = 2 * num_layers
outputs, hidden_states = model(inputs, hidden_states)
hidden_states[:2,:,:] + hidden_states[2:,:,:]

tensor([[[ 0.5128, -0.3853,  0.1114, -0.3361, -0.1792,  0.7587, -0.1266,
          -0.0423, -0.1266,  0.4259]],

        [[ 0.3262, -0.4404, -0.6756, -0.6569,  0.2367,  0.3934, -0.8896,
           0.7124,  0.7376, -0.6325]]], grad_fn=<AddBackward0>)

In [ ]:
attn_weights = torch.randn((1, 3, 10))
mask = torch.tensor([1, 5, 6])

In [ ]:
B, num_heads, T, d = (3, 2, 3, 4)
queries = torch.randn((B,num_heads,T,d))
keys = torch.randn((B,num_heads,T,d))
batch_size = queries.size(0)
# queries has shape (B, m, T, d)
attn_weights = []
for b in range(B):
  head_weights = []
  for m in range(num_heads):
    x_t = queries[b,m] @ keys[b,m].t() # T x d
    x_t = x_t.unsqueeze(0)
    # print(x_t.size())
    head_weights.append(x_t)
  head_weights = torch.cat(head_weights, dim=0)
  head_weights = head_weights.unsqueeze(0)
  # print(head_weights.size())
  attn_weights.append(head_weights)
# len(attn_weights)  
attn_weights = torch.cat(attn_weights, dim=0)
# attn_weights.size()

In [ ]:
B, T, m, d = 5, 256, 10, 3
mask = torch.tensor([1,1,1,0,0])
x = torch.randn(5,5)
# x = x.masked_fill(mask.unsqueeze(-1)==0, 1e-9)
x[0].unsqueeze(-1).size()

torch.Size([5, 1])

In [19]:
seq_len = 23

x_t = torch.randn((seq_len, seq_len))
mask = torch.randn((1, seq_len))
mask = mask.repeat(seq_len, 1)
mask.size()

torch.Size([23, 23])

## Test cases

In [ ]:
## testing GRU
import torch
import torch.nn as nn
from encoder_decoder_solution import GRU

def test_GRU_forward():
  input_size = 5
  batch_size = 1
  sequence_length = 3
  hidden_size = 10
  inputs = torch.randn((batch_size, sequence_length, input_size))
  hidden_states = torch.randn((1, batch_size, hidden_size))

  gru = GRU(input_size, hidden_size)
  x, hidden_states = gru(inputs, hidden_states)

  gt_gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
  gt_x, gt_hidden_states = gt_gru(inputs, hidden_states)

  print("Your answers are: x has size {0}, hidden_states has size {1}.".format(x.size(), hidden_states.size()))
  print("Ground truths are: x has size {0}, hidden_states has size {1}.".format(gt_x.size(), gt_hidden_states.size()))

test_GRU_forward()

In [ ]:
from encoder_deoder_solution import EncoderDecoder

def test_encoder():
  batch_size = 5
  sequence_length = 512
  hidden_size = 256
  num_layers = 1
  inputs = torch.randn((batch_size, sequence_length, input_size))
  hidden_states = torch.randn((num_layers*2, batch_size, hidden_size))
  

In [ ]:
from encoder_decoder_solution import Attn

## testing Attn no masking 
batch_size = 5
num_layers = 1
input_size = 512
sequence_length = 256
hidden_size = 256
x_new = torch.randn((batch_size, sequence_length, input_size))
hidden_states = torch.randn((num_layers, batch_size, hidden_size))

a = Attn(hidden_size=hidden_size)
x, x_attn = a(x_new, hidden_states)



x has size: torch.Size([5, 256, 256])
hidden states has size: torch.Size([1, 5, 256])
Concatenated size is: torch.Size([5, 256, 512])


In [38]:
B, m, T, d = 7, 17, 23, 13 
shape = (B, T, m, d)

queries = torch.randn(shape)
keys = torch.randn(shape)
values = torch.randn(shape)

from transformer_solution import MultiHeadedAttention

multi_headed_attn = MultiHeadedAttention(d, m, T)
hidden_states = torch.randn((B, T, m * d))
outputs = multi_headed_attn.forward(hidden_states)

torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
torch.Size([17, 23, 13])
After merging: torch.Size([7, 23, 221])


In [ ]:
#@title 🔍 Quick look at the data { run: "auto" }
#@markdown Lets have quick look at a few samples in our test set.
n_samples_to_see = 3 #@param {type: "integer"}
for i in range(n_samples_to_see):
  print("-"*30)
  print("title:", dataset_test[i]["title"])
  print("content:", dataset_test[i]["content"])
  print("label:", dataset_test[i]["label"])

### 1️⃣ Tokenize the `text`
Tokenize the `text`portion of each sample (i.e. parsing the text to smaller chunks). Tokenization can happen in many ways; traditionally, this was done based on the white spaces. With transformer-based models, tokenization is performed based on the frequency of occurrence of "chunk of text". This frequency can be learned in many different ways. However the most common one is the [**wordpiece**](https://arxiv.org/pdf/1609.08144v2.pdf) model. 
> The wordpiece model is generated using a data-driven approach to maximize the language-model likelihood
of the training data, given an evolving word definition. Given a training corpus and a number of desired
tokens $D$, the optimization problem is to select $D$ wordpieces such that the resulting corpus is minimal in the
number of wordpieces when segmented according to the chosen wordpiece model.

Under this model:
1. Not all things can be converted to tokens depending on the model. For example, most models have been pretrained without any knowledge of emojis. So their token will be `[UNK]`, which stands for unknown.
2. Some words will be mapped to multiple tokens!
3. Depending on the kind of model, your tokens may or may not respect capitalization

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
#@title 🔍 Quick look at tokenization { run: "auto", vertical-output: true }
input_sample = "Welcome to IFT6135. We now teach you 🤗(HUGGING FACE) Library :DDD." #@param {type: "string"}
tokenizer.tokenize(input_sample)

NameError: ignored

### 2️⃣ Encoding
Once we have tokenized the text, we then need to convert these chuncks to numbers so we can feed them to our model. This conversion is basically a look-up in a dictionary **from `str` $\to$ `int`**. The tokenizer object can also perform this work. While it does so it will also add the *special* tokens needed by the model to the encodings. 

In [ ]:
#@title 🔍 Quick look at token encoding { run: "auto"}
input_sample = "Welcome to IFT6135. We now teach you 🤗(HUGGING FACE) Library :DDD." #@param {type: "string"}

print("--> Token Encodings:\n",tokenizer.encode(input_sample))
print("-."*15)
print("--> Token Encodings Decoded:\n",tokenizer.decode(tokenizer.encode(input_sample)))

--> Token Encodings:
 [101, 6160, 2000, 2065, 2102, 2575, 17134, 2629, 1012, 2057, 2085, 6570, 2017, 100, 1006, 17662, 2227, 1007, 3075, 1024, 20315, 2094, 1012, 102]
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
--> Token Encodings Decoded:
 [CLS] welcome to ift6135. we now teach you [UNK] ( hugging face ) library : ddd. [SEP]


### 3️⃣ Truncate/Pad samples
Since all the sample in the batch will not have the same sequence length, we would need to truncate the longer sequences (i.e. the ones that exeed a predefined maximum length) and pad the shorter ones so we that we can equal length for all the samples in the batch. Once this is achieved, we would need to convert the result to `torch.Tensor`s and return. These tensors will then be retrieved from the [dataloader](https://https//pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

In [ ]:
class Collate:
    def __init__(self, tokenizer: str, max_len: int) -> None:
        self.tokenizer_name = tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
        self.max_len = max_len

    def __call__(self, batch: List[Dict[str, Union[str, int]]]) -> Dict[str, torch.Tensor]:
        texts = list(map(lambda batch_instance: batch_instance["title"], batch))
        tokenized_inputs = self.tokenizer(
            texts,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
            return_token_type_ids=False,
        )
        
        labels = list(map(lambda batch_instance: int(batch_instance["label"]), batch))
        labels = torch.LongTensor(labels)
        return dict(tokenized_inputs, **{"labels": labels})

In [ ]:
#@title 🧑‍🍳 Setting up the collate function { run: "auto" }
tokenizer_name = "bert-base-uncased" #@param {type: "string"}
sample_max_length = 256 #@param {type:"slider", min:32, max:512, step:1}
collate = Collate(tokenizer=tokenizer_name, max_len=sample_max_length)

In [ ]:
from transformers import AutoModel
from encoder_decoder_solution import EncoderDecoder
from transformer_solution import Transformer
import torch.nn as nn
torch.random.manual_seed(0)

class ReviewClassifier(nn.Module):
    def __init__(self, backbone: str, backbone_hidden_size: int, nb_classes: int):
        super(ReviewClassifier, self).__init__()
        self.backbone = backbone
        self.backbone_hidden_size = backbone_hidden_size
        self.nb_classes = nb_classes
        self.back_bone = AutoModel.from_pretrained(
            self.backbone,
            output_attentions=False,
            output_hidden_states=False,
        )
        self.classifier = torch.nn.Linear(self.backbone_hidden_size, self.nb_classes)

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        back_bone_output = self.back_bone(input_ids, attention_mask=attention_mask)
        hidden_states = back_bone_output[0]
        pooled_output = hidden_states[:, 0]  # getting the [CLS] token
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits

class ReviewClassifierLSTM(nn.Module):
    def __init__(self, nb_classes: int, encoder_only: bool = False, dropout=0.5):
        super(ReviewClassifierLSTM, self).__init__()
        self.nb_classes = nb_classes
        self.encoder_only = encoder_only
        self.back_bone = EncoderDecoder(dropout=dropout, encoder_only=encoder_only) 
        self.classifier = torch.nn.Linear(256, self.nb_classes)
       
    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        hidden_states, _ = self.back_bone(input_ids, attention_mask)
        pooled_output = hidden_states 
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits


class ReviewClassifierTransformer(nn.Module):
    def __init__(self, nb_classes: int, num_heads: int = 4, num_layers: int = 4, block: str="prenorm", dropout: float = 0.3):
        super(ReviewClassifierTransformer, self).__init__()
        self.nb_classes = nb_classes
        self.back_bone = Transformer(num_heads=num_heads, num_layers=num_layers, block=block, dropout=dropout)
        self.classifier = torch.nn.Linear(256, self.nb_classes)

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, labels: Optional[torch.Tensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        back_bone_output = self.back_bone(input_ids, attention_mask)
        hidden_states = back_bone_output
        pooled_output = hidden_states
        logits = self.classifier(pooled_output)
        if labels is not None:
            loss_fn = torch.nn.CrossEntropyLoss()
            loss = loss_fn(
                logits.view(-1, self.nb_classes),
                labels.view(-1),
            )
            return loss, logits
        return logits

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"--> Device selected: {device}")
def train_one_epoch(
    model: torch.nn.Module, training_data_loader: DataLoader, optimizer: torch.optim.Optimizer, logging_frequency: int, testing_data_loader: DataLoader, logger: dict):
    model.train()
    optimizer.zero_grad()
    epoch_loss = 0
    logging_loss = 0
    start_time = time.time()
    mini_start_time = time.time()
    for step, batch in enumerate(training_data_loader):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        logging_loss += loss.item()

        if (step + 1) % logging_frequency == 0:
            freq_time = time.time()-mini_start_time
            logger['train_time'].append(freq_time+logger['train_time'][-1])
            logger['train_losses'].append(logging_loss/logging_frequency)
            print(f"Training loss @ step {step+1}: {logging_loss/logging_frequency}")
            eval_acc, eval_f1, eval_loss, eval_time = evaluate(model, testing_data_loader)
            logger['eval_accs'].append(eval_acc)
            logger['eval_f1s'].append(eval_f1)
            logger['eval_losses'].append(eval_loss)
            logger['eval_time'].append(eval_time+logger['eval_time'][-1])

            logging_loss = 0
            mini_start_time = time.time()

    return epoch_loss / len(training_data_loader), time.time()-start_time


def evaluate(model: torch.nn.Module, test_data_loader: DataLoader):
    model.eval()
    model.to(device)
    eval_loss = 0
    correct_predictions = {i: 0 for i in range(2)}
    total_predictions = {i: 0 for i in range(2)}
    preds = []
    targets = []
    start_time = time.time()
    with torch.no_grad():
        for step, batch in enumerate(test_data_loader):
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            eval_loss += loss.item()

            predictions = np.argmax(outputs[1].detach().cpu().numpy(), axis=1)
            preds.extend(predictions.tolist())
            targets.extend(batch["labels"].cpu().numpy().tolist())

            for target, prediction in zip(batch["labels"].cpu().numpy(), predictions):
                if target == prediction:
                    correct_predictions[target] += 1
                total_predictions[target] += 1
    accuracy = (100.0 * sum(correct_predictions.values())) / sum(total_predictions.values())
    f1 = f1_score(targets, preds)
    model.train()
    return accuracy, round(f1, 4), eval_loss / len(test_data_loader), time.time() - start_time
import json
def save_logs(dictionary, log_dir, exp_id):
  log_dir = os.path.join(log_dir, exp_id)
  os.makedirs(log_dir, exist_ok=True)
  # Log arguments
  with open(os.path.join(log_dir, "args.json"), "w") as f:
    json.dump(dictionary, f, indent=2)

nb_epoch = 100 
batch_size = 512
logging_frequency = 5 
learning_rate = 1e-5

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, collate_fn=collate)
for i in range(1, 7):
  experimental_setting = i
  # 4 experimental settings

  if experimental_setting == 1:
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=True)
  if experimental_setting == 2:
    model = ReviewClassifierLSTM(nb_classes=2, dropout=0.3, encoder_only=False)
  if experimental_setting == 3:
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=2, block='prenorm', dropout=0.3)
  if experimental_setting == 4:
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=4, block='prenorm', dropout=0.3)
  if experimental_setting == 5:
    model = ReviewClassifierTransformer(nb_classes=2, num_heads=4, num_layers=2, block='postnorm', dropout=0.3)
  if experimental_setting == 6: 
    model = ReviewClassifier(backbone="bert-base-uncased", backbone_hidden_size=768, nb_classes=2)
    for parameter in model.back_bone.parameters():
      parameter.requires_grad= False
    logging_frequency = 703


  # setting up the optimizer
  optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, eps=1e-8)
  model.to(device)
  logger = dict()
  logger['train_time'] = [0]
  logger['eval_time'] = [0]
  logger['train_losses'] = []
  logger['eval_accs'] = []
  logger['eval_f1s'] = []
  logger['eval_losses'] = []
  
  logger['parameters'] = sum([p.numel() for p in model.back_bone.parameters() if p.requires_grad])
  nb_epoch=1

  train_loss, train_time = train_one_epoch(model, train_loader, optimizer, logging_frequency, test_loader, logger)
  eval_acc, eval_f1, eval_loss, eval_time  = evaluate(model, test_loader)
  logger["total_train_loss"] = train_loss
  logger["total_train_time"] = train_time
  logger["final_eval_loss"] = eval_loss
  logger["final_eval_time"] = eval_time
  logger["final_eval_acc"] = eval_acc
  logger["final_eval_f1"] = eval_f1
  logger['train_time'] = logger['train_time'][1:]
  logger['eval_time'] = logger['eval_time'][1:]

  print(f"    Epoch: {1} Loss/Test: {eval_loss}, Loss/Train: {train_loss}, Acc/Test: {eval_acc}, F1/Test: {eval_f1}, Train Time: {train_time}, Eval Time: {eval_time}")
  save_logs(logger, "assignment/log", str(experimental_setting))


In [ ]:
import torch


torch.nn.GRU